In [10]:
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot

import maestro_file
from pathlib import Path

from bokeh.io import output_notebook
from bokeh.plotting import figure, show

import re
from concurrent.futures import ProcessPoolExecutor

output_notebook()

Loading BokehJS ...

In [14]:
# path = '/home/barak/data/ComDePri-EWS/pickles/vernons_saccades_coil_data_with_speed_and_proj.pkl'
# vernons_saccades_coil_data = pd.read_pickle(path)
# vernons_saccades_coil_data['task'] = 'Manu_saccade'
# print(vernons_saccades_coil_data.head())
# vernons_saccades_coil_data.to_pickle(path)

In [3]:
base_path = Path('/home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/')
dirs = [d for d in base_path.iterdir() if d.is_dir()]

files = []
for dir_path in dirs:
    files += list(f for f in dir_path.iterdir() if f.is_file())

len(dirs), len(files)

(73, 86417)

In [ ]:
def get_file_data(file_path: Path) -> dict:
    posNormalizer = 40
    velNormalizer = 10.8826
    with open(file_path, 'rb') as f:
        data_file = maestro_file.DataFile.load(f.read(), file_path.name)

    if not re.match(r'^d\d+$', data_file.trial.name):
        return {}
    
    trail_row = {
        'filename_name': data_file.file_name,
        'trail_session': data_file.file_name.split('.')[0],
        'trail_number': data_file.file_name.split('.')[1],
        'trial_name': data_file.trial.name,
        'hPos': np.array(data_file.ai_data[0]) / posNormalizer,
        'vPos': np.array(data_file.ai_data[1]) / posNormalizer,
        'hVel': np.array(data_file.ai_data[2]) / velNormalizer,
        'vVel': np.array(data_file.ai_data[3]) / velNormalizer,
        'motion_onset': 1000,
        'cue_onset': 1000,
        'trial_length': len(data_file.ai_data[0]),
        'dir': int(data_file.trial.name[1:]),
        'blinks': data_file.blinks,
        'task': 'Manu_saccade'
    }
    return trail_row

file_path = Path('/home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/4-11-2015/Ve041115.0351')
test_data = get_file_data(file_path)

In [8]:
def process_file(file):
    try:
        file_data = get_file_data(file)
        if file_data == {}:
            return None
        else:
            return file_data
    except Exception as e:
        print(f'Error processing file {file}: {e}')
        return None

with ProcessPoolExecutor() as executor:
    results = list(executor.map(process_file, files))

# Filter out None results
results = [result for result in results if result is not None]

# Create a DataFrame from the results
files_df = pd.DataFrame(results)
files_df

Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/10-20-2014/.DS_Store: Maestro data file size in bytes (6148) is not a multiple of 1024!
Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/8-26-2015/Archive.zip: Maestro data file size in bytes (1368086) is not a multiple of 1024!
Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/9-22-2014/.DS_Store: Maestro data file size in bytes (6148) is not a multiple of 1024!
Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/4-7-2015/savethis.m: Maestro data file size in bytes (877) is not a multiple of 1024!
Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/4-7-2015/.DS_Store: Maestro data file size in bytes (6148) is not a multiple of 1024!
Error processing file /home/barak/data/ComDePri-EWS/data/data_from_Mati/Vernon_Neural/4-7-2015/untitled.fig: Maestro data file size

,filename_name,trail_session,trail_number,trial_name,hPos,vPos,hVel,vVel,motion_onset,cue_onset,trial_length,dir,blinks
0,Ve110414.1269,Ve110414,1269,d90,"[0.325, 0.35, 0.35, 0.35, 0.325, 0.35, 0.35, 0...","[0.975, 0.975, 0.975, 0.975, 0.975, 0.975, 0.9...","[0.4594490287247533, 0.36755922297980265, 0.45...","[-0.18377961148990132, -0.18377961148990132, -...",1000,1000,2161,90,None
1,Ve110414.1257,Ve110414,1257,d0,"[-0.025, -0.025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-4.0, -4.025, -4.0, -4.025, -4.025, -4.0, -4....","[5.880947567676842, 5.7890577619318915, 5.6052...","[-1.1945674746843584, -1.286457280429309, -1.1...",1000,1000,2161,0,None
2,Ve110414.0116,Ve110414,0116,d135,"[10.7, 10.7, 10.7, 10.7, 10.7, 10.7, 10.7, 10....","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...","[0.0, 0.0, 0.18377961148990132, 0.275669417234...","[-0.7351184459596053, -0.8270082517045559, -0....",1000,1000,2161,135,None
3,Ve110414.1235,Ve110414,1235,d180,"[6.35, 6.375, 6.4, 6.425, 6.45, 6.525, 6.55, 6...","[6.625, 6.55, 6.425, 6.275, 6.1, 5.9, 5.675, 5...","[9.005200963005164, 9.097090768750116, 9.28087...","[-13.048352415782993, -19.113079594949735, -27...",1000,1000,2161,180,None
4,Ve110414.0137,Ve110414,0137,d180,"[10.25, 10.25, 10.275, 10.25, 10.25, 10.25, 10...","[-0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0....","[-0.09188980574495066, -0.09188980574495066, 0...","[0.0, 0.09188980574495066, 0.18377961148990132...",1000,1000,2161,180,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12824,Ve092414.1202,Ve092414,1202,d225,"[2.5, 2.45, 2.4, 2.35, 2.275, 2.2, 2.15, 2.075...","[-1.125, -1.1, -1.075, -1.075, -1.05, -1.025, ...","[-59.26892470549317, -57.79868781357396, -56.9...","[27.10749269476044, 26.004815025821035, 24.810...",1000,1000,2161,225,None
12825,Ve092414.0784,Ve092414,0784,d180,"[2.3, 2.3, 2.325, 2.3, 2.3, 2.325, 2.3, 2.325,...","[-0.9, -0.9, -0.925, -0.925, -0.925, -0.925, -...","[3.767482035542977, 3.5837024240530755, 3.3999...","[-1.8377961148990132, -1.7459063091540625, -1....",1000,1000,2161,180,None
12826,Ve092414.1121,Ve092414,1121,d315,"[8.9, 8.9, 8.9, 8.9, 8.875, 8.9, 8.9, 8.875, 8...","[0.225, 0.225, 0.225, 0.225, 0.225, 0.2, 0.225...","[1.7459063091540625, 1.7459063091540625, 2.021...","[-0.5513388344697039, -0.5513388344697039, -0....",1000,1000,2161,315,None
12827,Ve092414.1218,Ve092414,1218,d45,"[0.175, -0.15, -0.5, -0.875, -1.225, -1.6, -2....","[-2.6, -2.375, -2.15, -1.95, -1.725, -1.5, -1....","[-173.48795324646684, -173.85551246944664, -17...","[183.87150129564625, 186.62819546799477, 188.0...",1000,1000,2161,45,None


In [9]:
files_df.to_pickle('/home/barak/data/ComDePri-EWS/pickles/vernon_data.pkl')